### **Inputs**

In [634]:
BuyDip = -200 #float(input("Sell if % Decrease Over(negative decimal input): "))
SellProfit = 200 #float(input("Sell if % Increase Over(decimal input): "))
BuyChange = float(input("Buy if % Decrease Over(negative decimal input): "))
SellChange = float(input("Sell if % Increase Over(decimal input): "))
days = int(input("Days to take avg over: "))

Buy if % Decrease Over(negative decimal input):  -.01
Sell if % Increase Over(decimal input):  0.01
Days to take avg over:  5


### **data organization and initial analysis**

In [636]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# importing data set
df = pd.read_csv('Bitcoin Historical Data.csv')

# fixing dataset so that it can be interpreted by the model
df['Price'] = df['Price'].str.replace(',','')
df['Open'] = df['Open'].str.replace(',','')
df['High'] = df['High'].str.replace(',','')
df['Low'] = df['Low'].str.replace(',','')
df['Change %'] = df['Change %'].str.replace('%', '')

df['Price'] = df['Price'].astype(float)
df['Open'] = df['Open'].astype(float)
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Change %'] = df['Change %'].astype(float)

df['Vol.'] = df['Vol.'].str.replace(',','')
df['Vol.'] = df['Vol.'].str.replace('K','')
df['Vol.'] = df['Vol.'].str.replace('M','')
df['Vol.'] = df['Vol.'].str.replace('B','')
df['Vol.'] = df['Vol.'].astype(float)
df['Vol.'] = df['Vol.'] * 1000

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

df = df.reset_index(drop=True)

df['Fake_Index'] = np.array([i for i in range(0,len(df))])

df['Change %'] = df['Change %'] / 100

#df['Fake_Index'] = np.array([i for i in range(5,len(df))])
#calculate 5-day moving avg
df['MA5D'] = df['Price'].rolling(window=days).mean()

buy_sell = []
position = 0

for i in range(len(df)):
    if i < days-1:
        # For the first few rows where SMA is not available
        buy_sell.append('Pass')
        continue
        
#Make sure the buy fits within the 5-day moving avg
for i in range(days-1, len(df)):
    price = df.loc[i, 'Price']
    sma5 = df.loc[i, 'MA5D']

    avg = ((price - sma5)/sma5)
    #print(avg)
    print(price)

    
    if pd.isna(sma5):
        buy_sell.append('Pass')
        continue
        
    # logic to determine whether a user should have bought or sold on this given day, originally derived later in code
    if position == 0 and (avg < BuyChange):
        position = 1 # Buy Bitcoin
        #print((price - sma5)/sma5)
        buy_sell.append(('Buy'))
    elif position > 0 and (avg > SellChange):  
        position = 0 # Sell Bitcoin
        #print((price - sma5)/sma5)
        buy_sell.append(('Sell'))
    else:
        buy_sell.append('Pass') #hold do nothing

df['BuySell'] = buy_sell
df = df[['Date', 'Open', 'High', 'Low', 'Vol.', 'MA5D', 'Price', 'BuySell']]

922.9
919.3
941.2
913.5
894.2
905.7
954.8
956.0
962.2
951.0
944.2
916.5
961.0
1007.0
943.5
932.7
925.7
941.4
938.8
940.4
953.3
931.7
926.6
904.5
828.9
695.7
648.8
659.6
582.6
579.0
531.1
451.4
427.7
371.1
299.8
272.3
294.0
261.5
111.9
111.6
255.6
310.2
174.0
135.8
593.1
596.5
573.9
557.4
544.6
662.2
661.8
658.3
648.3
616.7
609.2
627.0
613.0
608.7
619.4
625.0
614.0
624.0
619.1
606.4
598.9
595.0
566.0
549.6
564.3
551.2
567.6
562.9
562.5
460.5
482.6
477.1
446.9
444.7
463.5
424.4
436.3
444.4
456.6
455.7
462.4
466.6
457.3
384.6
435.1
437.6
427.6
478.4
526.4
537.0
502.0
485.5
506.0
498.5
495.2
487.9
487.3
500.3
464.5
457.9
443.2
441.9
446.1
445.6
460.1
453.7
438.6
436.0
430.9
432.2
446.5
441.1
445.5
451.9
433.2
438.2
433.9
442.6
448.1
453.6
452.8
450.1
443.9
484.6
488.8
515.4
518.1
520.5
570.4
582.0
570.1
570.8
564.4
612.3
627.9
649.8
655.5
669.4
638.9
660.2
650.4
651.7
651.0
644.3
655.6
643.0
597.1
610.3
583.4
582.3
599.1
605.1
606.2
600.2
590.8
597.0
603.6
591.2
588.8
568.5
582.7
602.2
597

### **modeling**

In [452]:
X = df[['Fake_Index', 'Open', 'High', 'Low', 'Vol.', 'Change %']] # features (can be changed, r^2 doesn't change much though)
y = df['Price'] # target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.50)

model = LinearRegression() # linearity of dataset made it pretty easy to use a linear model to predict the prices just based on the opening and volume that day
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(r2_score(y_test, y_pred))

0.9993951587367179


In [458]:
y_pred = pd.Series(y_pred)
y_test = pd.Series(y_test)

train_data = pd.concat([X_train,y_train], axis=1)
train_data.drop(columns='Fake_Index', inplace=True)

orig_index = y_test.index.to_list()
y_test.reset_index(drop=True, inplace=True)

results = pd.concat([y_pred,y_test], axis=1, ignore_index=True)
results['Index'] = orig_index
results.set_index('Index', inplace=True)

results_new = pd.concat([results,X_test], axis=1)
results_new.rename(columns={0:'Predicted'}, inplace=True)
results_new.rename(columns={1:'Actual'}, inplace=True)

final = pd.merge(results_new, df['Date'], left_index=True, right_index=True, how='inner')
final.drop(columns='Fake_Index', inplace=True)

### **logic to determine whether to buy or sell**

In [460]:
buy_sell_actual = []

for i in range(len(final)):
    if i < 5:
        # For the first few rows where SMA is not available
        buy_sell_actual.append('Pass')
        continue
        
#Make sure the buy fits within the 5-day moving avg
    
    if pd.isna(sma5d):
        buy_sell_actual.append('Pass')
        continue
        
    # logic to determine whether a user should have bought or sold on this given day, originally derived later in code
    if position == 0 and (final['Predicted'].iloc[i] > final['Open']) and (((price - sma5d) / sma5d) <= BuyChange):
        position = 1 # Buy Bitcoin
        buy_sell_actual.append('Buy')
    elif position == 0 and (final['Actual'].iloc[i] > final['Open']) and (((price - sma5d) / sma5d) <= BuyChange):
        position = 1 # Buy Bitcoin
        buy_sell_actual.append('Buy')
    elif position > 0 and (((price - sma5d) / sma5d) >= SellChange):  
        position = 0 # Sell Bitcoin
        buy_sell_actual.append('Sell')
    else:
        buy_sell_actual.append('Pass') #hold do nothing
        
#for i in range(len(final)):
    #if (final['Predicted'].iloc[i] > final['Open'].iloc[i]) and (abs(final['Predicted'].iloc[i] - final['Open'].iloc[i]) / final['Open'].iloc[i] > SellChange):
        #buy_sell_pred.append('sell')
    #else:
       # buy_sell_pred.append('buy')
    
    #if final['Actual'].iloc[i] > final['Open'].iloc[i] and (abs(final['Predicted'].iloc[i] - final['Open'].iloc[i]) / final['Open'].iloc[i] > SellChange):
       # buy_sell_actual.append('sell')
    #else:
        #buy_sell_actual.append('buy')
final['BuySell'] = buy_sell_actual

final = final[['Date', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'BuySell']]
df = df[['Date', 'Open', 'High', 'Low', 'Vol.', 'MA5D', 'BuySell']]
#final.rename(columns={'Predicted':'Price'}, inplace=True)
#final.rename(columns={'BuySell_actual':'BuySell'}, inplace=True)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### **Data Organization**

In [638]:
final
final.to_csv("main_data.csv", index = False)
df.to_csv("test_data.csv", index = False)